In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from vehicle_list_data import get_vehicles_charging, get_node_totals
import vehicle_list_data
import pickle
import numpy as np

# Electricity Data

In [2]:
elec_df = pd.read_csv("data_test/Demand_for_California_(region)_hourly_-_UTC_time.csv", skiprows=5, names=["time", "MWH"])
elec_df["utc_time"] = pd.to_datetime(elec_df["time"])
elec_df["local_time"] = elec_df["utc_time"] + pd.Timedelta(hours=-8)

In [ ]:
# uncomment the vis you want
# plt.plot(elec_df["MWH"]) # plot entire series
# plt.plot(elec_df[elec_df["local_time"].dt.year==2021]["MWH"]) # plot specific year


# summer_months = [6,7] 
# plt.plot(elec_df[elec_df["local_time"].dt.month.isin(summer_months) & 
#                  elec_df["local_time"].dt.year.isin([2021])]["MWH"]) # plot specific months and years

# elec_df.groupby([elec_df['local_time'].dt.hour, elec_df['local_time'].dt.month]).mean().loc[0] #groupby hours, months

# plot all hourly distributions over months in 2021
elec_df_2021 = elec_df[elec_df['local_time'].dt.year==2021]
fig, axs = plt.subplots(3, 4, sharey=True, figsize=(15,10))
for r in range(3):
    for c in range(4):
        axs[r][c].set_ylabel("MWH")
        axs[r][c].set_xlabel("hour of day")
        axs[r][c].set_title("Hourly distribution: " + datetime.datetime.strptime(str(r*4+c+1), "%m").strftime("%b"))
        axs[r][c].plot(elec_df_2021.groupby([elec_df_2021['local_time'].dt.month, elec_df_2021['local_time'].dt.hour]).mean().loc[r*4+c+1]["MWH"])
fig.tight_layout()

# Volume GIF

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.animation import FuncAnimation
import pickle

from vehicle_list_data import get_index_edge_totals

In [ ]:
############## SET THESE PARAMETERS FIRST ##############
file = "trials/full_day_results/full_day_scenario4_algorithm_7_05_03_2022_22_09_07.pkl"
with open(file, 'rb') as inp:
    sim = pickle.load(inp)
name = "animation_full_day_scenario_4_alg_7"

In [ ]:
G = sim.station_demand_g
edge_totals = get_index_edge_totals(sim)

In [ ]:
def set_visual_node_attributes(G):
    for node in G.nodes:
        G.nodes[node]["color"] = "blue"
        G.nodes[node]["size"] = 5
    for node in nx.get_node_attributes(G,'demand'):
        G.nodes[node]["color"] = "green"
        G.nodes[node]["size"] = 20
    return G
G = set_visual_node_attributes(G)

In [ ]:
def animate(sim_index):
    node_color = nx.get_node_attributes(G,'color').values() 
    node_size = {node: edge_totals[sim_index][(node, node)] if (node, node) in edge_totals[sim_index] else G.nodes[node]["size"] for node in G.nodes}
    widths = {edge:edge_totals[sim_index][edge]/50 if edge in edge_totals[sim_index] else .002 for edge in G.edges}
    pos = nx.get_node_attributes(G,'pos')
    graph_viz = nx.draw(G, pos = pos, node_size = list(node_size.values()), width= list(widths.values()), node_color=node_color, arrows=False)

In [ ]:
fig = plt.figure(dpi=200)
node_color = nx.get_node_attributes(G,'color').values() 
node_size = {node: edge_totals[sim_index][(node, node)] if (node, node) in edge_totals[sim_index] else G.nodes[node]["size"] for node in G.nodes}
widths = {edge:edge_totals[sim_index][edge]/50 if edge in edge_totals[sim_index] else .002 for edge in G.edges}
pos = nx.get_node_attributes(G,'pos')
graph_viz = nx.draw(G, pos = pos, node_size = list(node_size.values()), width= list(widths.values()), node_color=node_color, arrows=False)

# Animator call
anim = animation.FuncAnimation(fig, animate, frames=24, interval=100, blit=False)
# anim.save('/tmp/animation.gif', writer='imagemagick', fps=30)
anim.save("gifs/" + name + ".gif", fps=5, writer="avconv", codec="libx264")

# Electricity Metric

In [3]:
############## SET THESE PARAMETERS FIRST ##############
file = "trials/wednesday_debug_2/wednesday_debug_2_scenario0_algorithm_2_05_04_2022_17_51_26.pkl"
with open(file, 'rb') as inp:
    sim = pickle.load(inp)

In [30]:
vehicle_list_data.get_electricity_metric(sim)

0.08333333333333333